In [1]:
# pip install torch==2.3.0 torchvision==0.18.0 tensorboard==2.16.2 scikit-learn==1.5.0 pandas==2.2.2 tqdm==4.66.4

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [3]:
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [4]:
from torch.utils.data import Dataset, DataLoader

In [5]:
import os

from PIL import Image
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# 1. Prepare data

- Step 1: Read image from hard drive
- Step 2: Transform image
- Step 3: Prepare image label

In [6]:
class MyBrainTumorDataset(Dataset):
    def __init__(self, data_folder, csv_path):
        self.data_folder = data_folder
        self.image_names = [name for name in os.listdir(data_folder) if name.endswith('.jpg')]

        # Transformation => Step 2
        self.transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        # Label => Step 3
        self.label_df = pd.read_csv(csv_path, usecols=['image_name', 'label'])
        self.image_names = [name for name in self.image_names if name in self.label_df.image_name.to_list()]
        
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]

        # Step 1: Read image from hard drive
        image = Image.open(os.path.join(self.data_folder, image_name))

        # Step 2: Transform image
        transformed_image = self.transform(image)

        # Step 3: Prepare image label
        label_str = self.label_df[self.label_df.image_name == image_name]['label'].values[0]
        if label_str == 'tumor':
            label = torch.tensor([0, 1], dtype=torch.float32)
        else:
            label = torch.tensor([1, 0], dtype=torch.float32)

        return transformed_image, label

In [7]:
train_dataset = MyBrainTumorDataset(
    data_folder='data/brain_tumor_dataset/train',
    csv_path='data/brain_tumor_dataset/brain_multi.csv'
)

In [8]:
len(train_dataset)

5744

In [9]:
train_dataset[0]

(tensor([[[-0.5686, -0.7961, -0.9922,  ..., -0.9922, -0.8196, -0.3804],
          [-0.9373, -0.9686, -0.9922,  ..., -0.9765, -0.8353, -0.3647],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9686, -0.8039, -0.3412],
          ...,
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922]],
 
         [[-0.5686, -0.7961, -0.9922,  ..., -0.9922, -0.8196, -0.3804],
          [-0.9373, -0.9686, -0.9922,  ..., -0.9765, -0.8353, -0.3647],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9686, -0.8039, -0.3412],
          ...,
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
          [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922]],
 
         [[-0.5686, -0.7961, -0.9922,  ..., -0.9922, -0.8196, -0.3804],
          [-0.9373, -0.9686,

In [10]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)
train_dataloader

# 2. Build model

In [11]:
class MyCustomCNN(nn.Module):
    def __init__(self):
        super(MyCustomCNN, self).__init__()

        # Image features extractor: Embed image into vector
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu_3 = nn.ReLU()
        self.pool_3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv_4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu_4 = nn.ReLU()
        self.pool_4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Classification
        self.linear_1 = nn.Linear(128 * 8 * 8, 128)
        self.relu_5 = nn.ReLU()
        self.linear_2 = nn.Linear(128, 2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.conv_1(x)
        x = self.relu_1(x)
        x = self.pool_1(x)

        x = self.conv_2(x)
        x = self.relu_2(x)
        x = self.pool_2(x)

        x = self.conv_3(x)
        x = self.relu_3(x)
        x = self.pool_3(x)

        x = self.conv_4(x)
        x = self.relu_4(x)
        x = self.pool_4(x)

        x = x.view(-1, 128 * 8 * 8)
        x = self.linear_1(x)
        x = self.relu_5(x)

        x = self.linear_2(x)
        x = self.softmax(x)
        return x

In [12]:
# log_name = 'my_custom_cnn'
# model = MyCustomCNN()
# model

In [13]:
# model.cuda()

In [14]:
class MyResNetCNN(nn.Module):
    def __init__(self):
        super(MyResNetCNN, self).__init__()
        
        self.backbone = models.resnet18(pretrained=True)
        num_features = self.backbone.fc.in_features
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-1])

        self.linear = nn.Linear(num_features, 2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.backbone(x)
        x = x.squeeze()
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [15]:
log_name = 'my_resnet_cnn'
model = MyResNetCNN()
model

/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResNetCNN(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [16]:
# model.cuda()

# 3. Train model

## 3.1. Init loss function and optimizer

In [17]:
loss_func = nn.CrossEntropyLoss()
loss_func

CrossEntropyLoss()

In [18]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

## 3.2. Train and evaluate model

In [19]:
writer = SummaryWriter(log_dir=f'train_logs/{log_name}')
writer

In [20]:
num_epoch = 10

In [21]:
model

MyResNetCNN(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [22]:
ckpt_folfer = f'ckpt/{log_name}'
os.makedirs(ckpt_folfer, exist_ok=True)

In [23]:
for epoch in range(num_epoch):
    # Train model
    model.train()
    for iteration_, (image, label) in enumerate(tqdm(train_dataloader, total=len(train_dataloader))):
        
        # Bước 1: Optimizer zero grad
        optimizer.zero_grad()

        # Bước 2: Foward data to model
        # image.cuda()
        pred = model(image)

        # Bước 3: Tính giá trị loss
        loss_value = loss_func(pred, label)

        # Bước 4: Cập nhật trọng số của mô hình
        loss_value.backward()
        optimizer.step()
        
        global_iteration = epoch * len(train_dataloader) + iteration_
        writer.add_scalar('train_loss_iter', loss_value, global_iteration)

    # Bước 5: (Tuỳ chọn) In các thông số ra ngoài màn hình
    print(f'Epoch={epoch}', f'Training loss={loss_value.item()}')
    writer.add_scalar('train_loss_epoch', loss_value, epoch)

    # Evaluate model
    model.eval()
    with torch.no_grad():
        loss_sum = 0
        pred_list, label_list = [], []
        for image, label in tqdm(train_dataloader, total=len(train_dataloader)):
            # image.cuda()
            pred = model(image)
            loss = loss_func(pred, label)
            loss_sum += loss.item()

            pred_list.append(pred)
            label_list.append(label)

        print(f'Test loss {loss_sum / len(train_dataloader)}')
        writer.add_scalar('test_loss_epoch', loss_value, epoch)
        
        # Calculate metrics
        final_pred = torch.concat(pred_list)
        final_pred = torch.argmax(final_pred, axis=1)
        final_label = torch.concat(label_list)
        final_label = torch.argmax(final_label, axis=1)
        

        epoch_accuracy_score = accuracy_score(final_pred, final_label)
        writer.add_scalar('test_accuracy_score_epoch', epoch_accuracy_score, epoch)

        epoch_precision_score = precision_score(final_pred, final_label)
        writer.add_scalar('test_precision_score_epoch', epoch_precision_score, epoch)

        epoch_recall_score = recall_score(final_pred, final_label)
        writer.add_scalar('test_recall_score_epoch', epoch_recall_score, epoch)

        epoch_f1_score = f1_score(final_pred, final_label)
        writer.add_scalar('test_f1_score_epoch', epoch_f1_score, epoch)

        print(classification_report(final_pred, final_label))
        
        torch.save(model.state_dict(), os.path.join(ckpt_folfer, f'ckpt_{epoch}.pth'))

  0%|                                                                                                                           | 0/90 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:54<00:00,  1.65it/s]


Epoch=0 Training loss=0.42942550778388977


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:35<00:00,  2.53it/s]


Test loss 0.3989162458313836
              precision    recall  f1-score   support

           0       0.71      0.89      0.79       974
           1       0.98      0.93      0.95      4770

    accuracy                           0.92      5744
   macro avg       0.84      0.91      0.87      5744
weighted avg       0.93      0.92      0.92      5744



  0%|                                                                                                                           | 0/90 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:53<00:00,  1.67it/s]


...

Test loss 0.31687549816237554
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1198
           1       1.00      1.00      1.00      4546

    accuracy                           1.00      5744
   macro avg       0.99      1.00      1.00      5744
weighted avg       1.00      1.00      1.00      5744



  0%|                                                                                                                           | 0/90 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:52<00:00,  1.72it/s]


Epoch=9 Training loss=0.35087767243385315


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:36<00:00,  2.48it/s]

Test loss 0.3166029897001055
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1200
           1       1.00      1.00      1.00      4544

    accuracy                           1.00      5744
   macro avg       0.99      1.00      1.00      5744
weighted avg       1.00      1.00      1.00      5744



# 4. Predict new data

In [24]:
state_dict = torch.load(f'{ckpt_folfer}/ckpt_3.pth', map_location='cpu')
state_dict

OrderedDict([('backbone.0.weight',
              tensor([[[[-9.6857e-03, -4.7254e-03, -6.3265e-04,  ...,  5.7259e-02,
                          1.7740e-02, -1.1521e-02],
                        [ 1.0995e-02,  9.8585e-03, -1.0956e-01,  ..., -2.7139e-01,
                         -1.2897e-01,  4.1256e-03],
                        [-6.9556e-03,  5.9923e-02,  2.9637e-01,  ...,  5.1964e-01,
                          2.5666e-01,  6.3721e-02],
                        ...,
                        [-2.8172e-02,  1.6413e-02,  7.3423e-02,  ..., -3.3121e-01,
                         -4.1886e-01, -2.5642e-01],
                        [ 3.0515e-02,  4.2112e-02,  6.3945e-02,  ...,  4.1587e-01,
                          3.9579e-01,  1.6780e-01],
                        [-1.3596e-02, -2.3897e-03, -2.2350e-02,  ..., -1.4907e-01,
                         -8.0883e-02, -4.2570e-03]],
              
                       [[-1.0660e-02, -2.5206e-02, -3.3464e-02,  ...,  3.3164e-02,
                          1

In [25]:
new_model_1 = MyCustomCNN()

In [26]:
new_model_2 = MyResNetCNN()

/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
new_model_2.load_state_dict(state_dict)

<All keys matched successfully>

In [28]:
new_model_1.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for MyCustomCNN:
	Missing key(s) in state_dict: "conv_1.weight", "conv_1.bias", "conv_2.weight", "conv_2.bias", "conv_3.weight", "conv_3.bias", "conv_4.weight", "conv_4.bias", "linear_1.weight", "linear_1.bias", "linear_2.weight", "linear_2.bias". 
	Unexpected key(s) in state_dict: "backbone.0.weight", "backbone.1.weight", "backbone.1.bias", "backbone.1.running_mean", "backbone.1.running_var", "backbone.1.num_batches_tracked", "backbone.4.0.conv1.weight", "backbone.4.0.bn1.weight", "backbone.4.0.bn1.bias", "backbone.4.0.bn1.running_mean", "backbone.4.0.bn1.running_var", "backbone.4.0.bn1.num_batches_tracked", "backbone.4.0.conv2.weight", "backbone.4.0.bn2.weight", "backbone.4.0.bn2.bias", "backbone.4.0.bn2.running_mean", "backbone.4.0.bn2.running_var", "backbone.4.0.bn2.num_batches_tracked", "backbone.4.1.conv1.weight", "backbone.4.1.bn1.weight", "backbone.4.1.bn1.bias", "backbone.4.1.bn1.running_mean", "backbone.4.1.bn1.running_var", "backb

In [29]:
new_model_2.eval()
with torch.no_grad():
    loss_sum = 0
    pred_list, label_list = [], []
    for image, label in tqdm(train_dataloader, total=len(train_dataloader)):
        pred = new_model_2(image)
        loss = loss_func(pred, label)
        loss_sum += loss.item()

        pred_list.append(pred)
        label_list.append(label)

    print(f'Test loss {loss_sum / len(train_dataloader)}')

    # Calculate metrics
    final_pred = torch.concat(pred_list)
    final_pred = torch.argmax(final_pred, axis=1)
    final_label = torch.concat(label_list)
    final_label = torch.argmax(final_label, axis=1)

    print(classification_report(final_pred, final_label))

  0%|                                                                                                                           | 0/90 [00:00<?, ?it/s]/Users/minhhuunguyen/REPOSITORY/python-practice/python_practice_env/lib/python3.9/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:36<00:00,  2.47it/s]

Test loss 0.328037499388059
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1178
           1       1.00      0.99      0.99      4566

    accuracy                           0.99      5744
   macro avg       0.98      0.99      0.98      5744
weighted avg       0.99      0.99      0.99      5744



In [30]:
# Test loss 0.47746641347625796
#               precision    recall  f1-score   support

#            0       0.13      0.91      0.23       173
#            1       1.00      0.81      0.89      5571

#     accuracy                           0.81      5744
#    macro avg       0.56      0.86      0.56      5744
# weighted avg       0.97      0.81      0.87      5744